In [0]:
!pip install torch torchvision
!pip install pytorch-ignite
!pip install torchsummary

In [0]:
from google.colab import drive
drive.mount('gdrive')

In [1]:
cd "/content/gdrive/My Drive/Colab Notebooks/Vision/pytorch-hair-segmentation"

/content/gdrive/My Drive/Colab Notebooks/Vision/pytorch-hair-segmentation


In [2]:
ls

ckpt/  logs/    models/    notebooks/  test.py  utils/
data/  main.py  networks/  README.md   tests/


In [0]:
import os

import torch
from torch import nn
import torchvision
device = 'cuda' if torch.cuda.is_available() else 'cpu'
assert device == 'cuda', "Change runtime type"

In [0]:
from networks.fcn_mobilenetv2 import FCN

In [0]:
model = FCN().to(device).train()
model.load_state_dict(torch.load('ckpt/FCN_SGD_epoch_43.pth')['weight'])

In [6]:
import torchsummary
torchsummary.summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [0]:
tensor = torch.randn(1,3,224,224).to(device)

In [9]:
out = model(tensor)
out

tensor([[[[ 0.0676,  0.1153,  0.1331,  ...,  0.0988,  0.0983,  0.0565],
          [ 0.1518,  0.0253,  0.1615,  ...,  0.1047,  0.0778, -0.0551],
          [ 0.1296,  0.0525,  0.0984,  ...,  0.1306,  0.0744, -0.0265],
          ...,
          [ 0.1026, -0.1322, -0.1095,  ...,  0.0200,  0.0669,  0.0244],
          [ 0.1147, -0.0047, -0.0778,  ...,  0.0689,  0.0482,  0.0248],
          [ 0.0694, -0.1155, -0.0766,  ...,  0.0646,  0.0326, -0.0232]]]],
       device='cuda:0', grad_fn=<CudnnConvolutionBackward>)

## Loading Figaro dataset using get_loader

In [0]:
# importing dataloader

from data import get_loader

# you have to predefine transforms to load dataset
# this transforms images and masks while loading
# example transforms

from utils import joint_transforms as jnt_trnsf
import torchvision.transforms as std_trnsf


# transforms that are applied to both images and masks
# includes geometrical changes like flip
# implemented in ./utils/joint_transforms.py
joint_transforms = jnt_trnsf.Compose([
    jnt_trnsf.Resize(256),
    jnt_trnsf.RandomRotate(5),
    jnt_trnsf.CenterCrop(224),
    jnt_trnsf.RandomHorizontallyFlip()
])


# transforms that are applied to only images
# this includes color jittering, normalizing, blurring, etc
# use torchvision.transforms, or implement additional transforms in 'utils'
train_image_transforms = std_trnsf.Compose([
    std_trnsf.ColorJitter(0.05, 0.05, 0.05, 0.05),
    std_trnsf.ToTensor(),
    std_trnsf.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


test_image_transforms = std_trnsf.Compose([
    std_trnsf.ToTensor(),
    std_trnsf.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# transforms that are applied to only masks
mask_transforms = std_trnsf.Compose([
    std_trnsf.ToTensor()
    ])

# predifine other needed arguments
batch_size = 32
num_workers = os.cpu_count()
data_dir = 'data/Figaro1k/'

In [0]:
train_loader = get_loader(dataset='figaro',
                          data_dir=data_dir,
                          train=True,
                          joint_transforms=joint_transforms,
                          image_transforms=train_image_transforms,
                          mask_transforms=mask_transforms,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=num_workers)

test_loader = get_loader(dataset='figaro',
                         data_dir=data_dir,
                         train=False,
                         image_transforms=test_image_transforms,
                         mask_transforms=mask_transforms,
                         batch_size=1,
                         shuffle=False,
                         num_workers=num_workers)

## Defining Optimizer & Scheduler & loss & device

In [0]:
# torch.optim
optimizer = torch.optim.SGD(model.parameters(), 
                             lr = 0.01,
                             momentum = 0.9
                            )

# torch.
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

# torch.nn
loss = torch.nn.BCEWithLogitsLoss()

## Using Pytorch Ignite

In [0]:
# ignite moduels
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss

# custom modules
from utils.metrics import Accuracy, MeanIU

# to measure time
import time, datetime

In [0]:
# trainer and evaluator
trainer = create_supervised_trainer(model, optimizer, loss, device=device)
evaluator = create_supervised_evaluator(model,
                                        metrics={
                                            'pix-acc': Accuracy(),
                                            'mean-iu': MeanIU(0.5),
                                            'loss': Loss(loss)
                                            },
                                        device=device)

In [0]:
# saving training state if you want
from utils import update_state, save_ckpt_file
state = update_state(model.state_dict(), 0, 0, 0, 0)

In [14]:
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(trainer):
    num_iter = (trainer.state.iteration - 1) % len(train_loader) + 1
    if num_iter % 20 == 0:
        print("Epoch[{}] Iter[{:03d}] Loss: {:.2f}".format(
            trainer.state.epoch, num_iter, trainer.state.output))

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    # evaluate training set
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    print("Training Results - Epoch: {}  Pix-acc: {:.3f} MeanIU: {:.3f} Avg-loss: {:.3f}".format(
        trainer.state.epoch, metrics['pix-acc'], metrics['mean-iu'], metrics['loss']))

    # update state
    update_state(model.state_dict(), metrics['loss'], state['val_loss'], state['val_pix_acc'], state['val_miu'])

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    # evaluate test(validation) set
    evaluator.run(test_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Pix-acc: {:.2f} MeanIU: {:.3f} Avg-loss: {:.2f}".format(
        trainer.state.epoch, metrics['pix-acc'], metrics['mean-iu'], metrics['loss']))

    # update scheduler
    scheduler.step(metrics['loss'])

    # update and save state
    ckpt_root = './ckpt/'
    filename = '{network}_{optimizer}_epoch_{epoch}.pth'
    ckpt_path = os.path.join(ckpt_root, filename)

    state = update_state(model.state_dict(), 0, metrics['loss'], metrics['pix-acc'], metrics['mean-iu'])
    path = ckpt_path.format(network='FCN',
                            optimizer='SGD',
                            epoch=trainer.state.epoch)
    save_ckpt_file(path, state)

t0 = time.time()
trainer.run(train_loader, max_epochs=50)
t1 = time.time()

print("Time taken: {}".format(str(datetime.timedelta(seconds=t1-t0))))

Epoch[1] Iter[020] Loss: 0.28
Training Results - Epoch: 1  Pix-acc: 0.895 MeanIU: 0.770 Avg-loss: 0.266
Validation Results - Epoch: 1  Pix-acc: 0.85 MeanIU: 0.183 Avg-loss: 0.38
Epoch[2] Iter[020] Loss: 0.30
Training Results - Epoch: 2  Pix-acc: 0.905 MeanIU: 0.738 Avg-loss: 0.232
Validation Results - Epoch: 2  Pix-acc: 0.87 MeanIU: 0.201 Avg-loss: 0.34
Epoch[3] Iter[020] Loss: 0.29
Training Results - Epoch: 3  Pix-acc: 0.904 MeanIU: 0.793 Avg-loss: 0.238
Validation Results - Epoch: 3  Pix-acc: 0.88 MeanIU: 0.166 Avg-loss: 0.33
Epoch[4] Iter[020] Loss: 0.31
Training Results - Epoch: 4  Pix-acc: 0.906 MeanIU: 0.792 Avg-loss: 0.236
Validation Results - Epoch: 4  Pix-acc: 0.87 MeanIU: 0.172 Avg-loss: 0.33
Epoch[5] Iter[020] Loss: 0.28
Training Results - Epoch: 5  Pix-acc: 0.908 MeanIU: 0.809 Avg-loss: 0.233
Validation Results - Epoch: 5  Pix-acc: 0.87 MeanIU: 0.170 Avg-loss: 0.35
Epoch[6] Iter[020] Loss: 0.29
Training Results - Epoch: 6  Pix-acc: 0.907 MeanIU: 0.787 Avg-loss: 0.234
Valida

KeyboardInterrupt: ignored

---
## Sanity check

In [0]:
import numpy as np
from PIL import Image

my_transforms = std_trnsf.Compose([
    std_trnsf.ToTensor(),
    std_trnsf.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

path = 'data/Figaro1k/Original/Training/Frame00001-org.jpg'
img = Image.open(path)
arr = np.array(img)

In [0]:
tensor = my_transforms(arr[166:390, 97:321])

In [0]:
out = model(tensor[None].to(device))

In [0]:
out_arr = out.detach().cpu().numpy()

In [0]:
from matplotlib import pyplot as plt